# Notebook de carga de dados do Diário Oficial da União no ElasticSearch

In [12]:
import pandas as pd
import json
import elasticsearch
from elasticsearch import helpers
from elasticsearch import Elasticsearch

ES_URL = "localhost:9200"
CAMINHO_ARQ_INFORMACAO = "./bpreg/informacao_dou.pip"

In [14]:
try:

    # Declare um cliente Elasticsearch
    client = elasticsearch.Elasticsearch(ES_URL)

    elastic_info = Elasticsearch.info(client)
    print ("Informação do Cluster:", json.dumps(elastic_info, indent=4 ))

except Exception as err:
    print ("Erro clinte Elasticsearch", err)
    client = None

Informação do Cluster: {
    "name": "elastic_node01",
    "cluster_name": "docker-cluster",
    "cluster_uuid": "Bui7DUIOSd-FSP9v4pfrlw",
    "version": {
        "number": "7.9.2",
        "build_flavor": "default",
        "build_type": "docker",
        "build_hash": "d34da0ea4a966c4e49417f2da2f244e3e97b4e6e",
        "build_date": "2020-09-23T00:45:33.626720Z",
        "build_snapshot": false,
        "lucene_version": "8.6.2",
        "minimum_wire_compatibility_version": "6.8.0",
        "minimum_index_compatibility_version": "6.0.0-beta1"
    },
    "tagline": "You Know, for Search"
}


### Carrega arquivo com informações do DOU

In [69]:
df_info_dou = pd.read_csv(CAMINHO_ARQ_INFORMACAO, delimiter="|")

In [70]:
df_info_dou.head()

,din_inclusao,dsc_objeto,texto,flg_importanciaassunto,flg_incluirsintese,nom_inclusao,dsc_tpobjeto
0,2018-02-02T08:56:10.597,Portaria SPE/MME nº 30 - Definir garantia físi...,Levamos ao conhecimento a Portaria SPE/MME nº ...,N,1,CARLA CARPI,PORTARIA
1,2018-02-02T08:59:18.183,Resolução Autorizativa ANEEL nº 6.808 - Revoga...,Levamos ao conhecimento a Resolução Autorizati...,N,1,CARLA CARPI,RESOLUÇÃO AUTORIZATIVA
2,2018-02-02T09:01:35.177,Resolução Autorizativa ANEEL nº 6.829 - Revoga...,Levamos ao conhecimento a Resolução Autorizati...,N,1,CARLA CARPI,RESOLUÇÃO AUTORIZATIVA
3,2018-02-02T09:03:36.753,Resolução Autorizativa ANEEL nº 6.830 - Revog...,Levamos ao conhecimento a Resolução Autorizati...,N,1,CARLA CARPI,RESOLUÇÃO AUTORIZATIVA
4,2018-02-02T09:06:48.003,Resolução Autorizativa ANEEL nº 6.844 - Autori...,Levamos ao conhecimento a Resolução Autorizati...,N,1,CARLA CARPI,RESOLUÇÃO AUTORIZATIVA


In [71]:
df_info_dou.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6339 entries, 0 to 6338
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   din_inclusao            6339 non-null   object
 1   dsc_objeto              6339 non-null   object
 2   texto                   6339 non-null   object
 3   flg_importanciaassunto  6339 non-null   object
 4   flg_incluirsintese      6339 non-null   int64 
 5   nom_inclusao            6339 non-null   object
 6   dsc_tpobjeto            6339 non-null   object
dtypes: int64(1), object(6)
memory usage: 346.8+ KB


### Define mapping para criação de índice

In [7]:
NOME_INDICE = 'informacao_dou'

# Gerando analizador customizado para tratar texto com tags html e decode oriundo diretamente do site da imprensa nacional
MAPA_INDICE = {
  "settings": {
      "number_of_shards":3,
      "analysis":{
          "analyzer":{
              "texto_analyzer": {
                  "type":"custom",
                  "tokenizer":"standard",
                  "char_filter": ["html_strip"],
                  "filter": ["asciifolding","lowercase"]
              }
          }
      }
  },
   "mappings":{
         "properties": {
            "din_inclusao":{
               "type":"date"
            },
            "dsc_objeto":{
               "type":"text",
               "fields":{
                     "raw":{
                        "type":"keyword"
                     }
               }
            },
            "texto":{
               "type":"text",
               "analyzer":"texto_analyzer",
               "fielddata":True,
               "fielddata_frequency_filter":{
                  "min":0.01,
                  "min_segment_size":10,
               }
            },
            "flg_importanciaassunto":{
               "type":"text"
            },
            "flg_incluirsintese":{
               "type":"text"
            },
            "nom_inclusao":{
               "type":"text"
               ,"fields":{
                  "raw":{
                     "type":"keyword"
                  }
               }
            },
            "dsc_tpobjeto":{
               "type":"text",
               "fields":{
                  "raw":{
                     "type":"keyword"
                  }
               }
            }
        }
     }
 }

### Cria indice com mapa customizado

In [73]:
# Apagar caso o indice exista
if client.indices.exists(NOME_INDICE):
    client.indices.delete(NOME_INDICE)
client.indices.create(NOME_INDICE, body=MAPA_INDICE)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'informacao_dou'}

### Inserção Bulk

In [65]:
# Não funcionou
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": NOME_INDICE,
                "_type": "_doc",
                "_source": document,
            }
    raise StopIteration

helpers.bulk(client, doc_generator(df_info_dou))

BulkIndexError: ('500 document(s) failed to index.', [{'index': {'_index': 'informacao_dou', '_type': '_doc', '_id': 'prrTP3YBNt7aN5wTuOzD', 'status': 400, 'error': {'type': 'mapper_parsing_exception', 'reason': 'failed to parse', 'caused_by': {'type': 'not_x_content_exception', 'reason': 'Compressor detection can only be called on some xcontent bytes or compressed xcontent bytes'}}, 'data': din_inclusao                                        2018-02-02T08:56:10.597
dsc_objeto                Portaria SPE/MME nº 30 - Definir garantia físi...
texto                     Levamos ao conhecimento a Portaria SPE/MME nº ...
flg_importanciaassunto                                                    N
flg_incluirsintese                                                        1
nom_inclusao                                                    CARLA CARPI
dsc_tpobjeto                                                       PORTARIA
Name: 0, dtype: object}}, {'index': {'_index': 'informacao_dou', '_type':

### Inserção manual RBAR

In [74]:
%%time
df_iter = df_info_dou.iterrows()
for index, document in df_iter:
    client.index(index=NOME_INDICE, body=document.to_dict())    

CPU times: user 10.7 s, sys: 820 ms, total: 11.5 s
Wall time: 1min 5s


In [75]:
# Verificando o status do indice
client.cluster.health(index=NOME_INDICE)  

{'cluster_name': 'docker-cluster',
 'status': 'yellow',
 'timed_out': False,
 'number_of_nodes': 1,
 'number_of_data_nodes': 1,
 'active_primary_shards': 3,
 'active_shards': 3,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 3,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 68.42105263157895}

In [76]:
# Alterando o número de replicações do indice para 0 -> Como só temos um nó não há replicação

client.indices.put_settings(index=NOME_INDICE, body={"number_of_replicas": 0})


{'acknowledged': True}

In [77]:
client.cluster.health(index=NOME_INDICE)  

{'cluster_name': 'docker-cluster',
 'status': 'green',
 'timed_out': False,
 'number_of_nodes': 1,
 'number_of_data_nodes': 1,
 'active_primary_shards': 3,
 'active_shards': 3,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 0,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 100.0}

### Verificando informações sobre o índice

In [17]:
client.indices.get(NOME_INDICE)

{'informacao_dou': {'aliases': {},
  'mappings': {'properties': {'din_inclusao': {'type': 'date'},
    'dsc_objeto': {'type': 'text', 'fields': {'raw': {'type': 'keyword'}}},
    'dsc_tpobjeto': {'type': 'text', 'fields': {'raw': {'type': 'keyword'}}},
    'flg_importanciaassunto': {'type': 'text'},
    'flg_incluirsintese': {'type': 'text'},
    'nom_inclusao': {'type': 'text', 'fields': {'raw': {'type': 'keyword'}}},
    'texto': {'type': 'text',
     'analyzer': 'texto_analyzer',
     'fielddata': True,
     'fielddata_frequency_filter': {'min': 0.01, 'min_segment_size': 10}}}},
  'settings': {'index': {'number_of_shards': '3',
    'provided_name': 'informacao_dou',
    'creation_date': '1607388840430',
    'analysis': {'analyzer': {'texto_analyzer': {'filter': ['asciifolding',
        'lowercase'],
       'char_filter': ['html_strip'],
       'type': 'custom',
       'tokenizer': 'standard'}}},
    'number_of_replicas': '0',
    'uuid': 'C7OGt9RsRImZ60H3_D0Ruw',
    'version': {'cr